In [2]:
import pandas as pd
import plotly.graph_objects as go
import json
import datetime
pd.options.mode.copy_on_write = True

## Duration trends

In [1]:
# This section of for pulling data directly from db

# with open('config/secrets.json') as f:
#     config = json.load(f)
# db = config['postgres']
# con = f"postgresql://{db['user']}:{db['password']}@{db['host']}/{db['dbname']}"
# stmt = '''
# select video_id, published_date, published_time, duration, vl.*
# from video_log vl  
# join (
# select * from video 
# where video_type='video') as v
# using(video_id)

# '''
# video = pd.read_sql(stmt,con)

In [15]:
# This section of for pulling data from sample_data
v = pd.read_csv('../sample_data/video.csv')
vl = pd.read_csv('../sample_data/video_log.csv')
v = v[v['video_type']=='video']
video = pd.merge(v[['video_id','published_date','duration']],vl,how='right',on='video_id')
del v,vl

In [16]:
video = video.drop_duplicates(subset='video_id',keep='last')
video['published_date'] = pd.to_datetime(video['published_date'])
video['created_date'] = pd.to_datetime(video['created_date'])
video = video[video['created_date']-video['published_date']>=datetime.timedelta(days=7)]
video['duration'] = pd.to_timedelta(video['duration']).dt.ceil('min')
video = video[video["duration"] < datetime.timedelta(hours=1)]


In [17]:
is_multi = video["duration"].value_counts() > 20
filtered = video[video["duration"].isin(is_multi[is_multi].index)]
filtered['duration'] = filtered.duration.dt.total_seconds()/60
bin_num = 20
filtered['time_binning'] = pd.cut(filtered['duration'],bins=bin_num,
                                  include_lowest=True,
                                  precision=0
                                #   labels=list(range(bin_num))
                                  )
bin_names = [f"{int(temp.left)}~{int(temp.right)}" for temp in filtered['time_binning'].dtype.categories]

In [19]:
trace1 = go.Bar(x = bin_names,
                     y=filtered.groupby('time_binning',observed=True)['duration'].count(),
                     name='Number of video',
                     yaxis='y1',
                     marker=dict(color='#1D1B86')
                     )
trace2 = go.Scatter(x=bin_names,
                      y=filtered.groupby('time_binning',observed=True)['view_count'].median().sort_index().values,
                      name='Video views median',
                      yaxis='y2',
                      mode='lines+markers',
                      marker_size=8
                      )
data = [trace1, trace2]
layout = go.Layout(title='Video performance base on duration (min)',
                    yaxis=dict(title='num'),
                    yaxis2=dict(title='views',
                                overlaying='y',
                                side='right'),
                    template="plotly_dark")
fig = go.Figure(data=data,layout=layout,)

fig.update_layout(legend=dict(
    y=1.15,
    x=0.75),
    autosize=False,
    width=800,
    height=500
    )

fig.show()

In [710]:
fig.write_image('images/duration_view_relation.png')
# fig.write_html('images/duration_view_relation.html')